In [0]:
from google.colab import drive
drive.mount('/vkcup/')

Drive already mounted at /vkcup/; to attempt to forcibly remount, call drive.mount("/vkcup/", force_remount=True).


In [0]:
!pip install catboost
!pip install xgboost
!pip install lightgbm

In [0]:
from lightgbm import LGBMRegressor

In [0]:
from catboost import CatBoostRegressor

In [0]:
import pandas as pd
import numpy as np

In [0]:
users = pd.read_csv("/vkcup/My Drive/Colab Notebooks/vkcup2/data/users.tsv", sep="\t")
users.head()

,user_id,sex,age,city_id
0,0,2,19,0
1,1,1,0,1
2,2,2,24,2
3,3,1,20,3
4,4,2,29,4


In [0]:
history = pd.read_csv("/vkcup/My Drive/Colab Notebooks/vkcup2/data/history.tsv", sep="\t")
history.head()

,hour,cpm,publisher,user_id
0,10,30.00,1,15661
1,8,41.26,1,8444
2,7,360.00,1,15821
3,18,370.00,1,21530
4,8,195.00,2,22148


In [0]:
validate = pd.read_csv("/vkcup/My Drive/Colab Notebooks/vkcup2/data/validate.tsv", sep="\t")
validate.head()

,cpm,hour_start,hour_end,publishers,audience_size,user_ids
0,220.0,1058,1153,"7,17",1906,"12,44,46,50,58,71,93,122,134,143,176,184,187,1..."
1,312.0,1295,1301,"3,18",1380,"29,81,98,102,165,167,195,205,218,231,242,263,3..."
2,70.0,1229,1249,"1,2,3,9,15,21",888,"12,23,25,29,45,85,92,124,156,190,272,334,456,5..."
3,240.0,1295,1377,"1,14",440,"44,122,187,209,242,255,312,345,382,465,513,524..."
4,262.0,752,990,"1,3,7,8",1476,"15,24,30,43,50,53,96,105,159,168,181,190,196,2..."


In [0]:
count_of_users_on_publ = []

from tqdm import tqdm_notebook
from datetime import datetime
import time

start_time = datetime.now()

coun = []
mean_cmp = []
max_cmp = []
min_cmp = []
for i in tqdm_notebook(validate.index):
  ids = [int(j) for j in validate['user_ids'].iloc[i].split(',')]
  publs = [int(j) for j in validate['publishers'].iloc[i].split(',')]
  df = history[(history['user_id'].isin(ids)) & (history['publisher'].isin(publs))]
  g = df.count()['hour']
  count_of_users_on_publ.append(g)

  mean_cmp.append(df['cpm'].mean())
  max_cmp.append(df['cpm'].max())
  min_cmp.append(df['cpm'].min())
  c = df[['publisher', 'user_id']].drop_duplicates().count()['publisher']
  coun.append(c)
  

In [0]:
validate_answers = pd.read_csv("/vkcup/My Drive/Colab Notebooks/vkcup2/data/validate_answers.tsv", sep="\t")
validate_answers.head()

,at_least_one,at_least_two,at_least_three
0,0.0430,0.0152,0.0073
1,0.0130,0.0000,0.0000
2,0.0878,0.0135,0.0000
3,0.2295,0.1295,0.0727
4,0.3963,0.2785,0.2270


In [0]:
import sys

import pandas as pd
import numpy as np


def load_answers(answers_filename):
    return pd.read_csv(answers_filename, sep="\t")


def get_smoothed_log_mape_column_value(responses_column, answers_column, epsilon):
    return np.abs(np.log(
        (responses_column + epsilon)
        / (answers_column + epsilon)
    )).mean()


def get_smoothed_mean_log_accuracy_ratio(answers, responses, epsilon=0.005):
    log_accuracy_ratio_mean = np.array(
        [
            get_smoothed_log_mape_column_value(responses.at_least_one, answers.at_least_one, epsilon),
            get_smoothed_log_mape_column_value(responses.at_least_two, answers.at_least_two, epsilon),
            get_smoothed_log_mape_column_value(responses.at_least_three, answers.at_least_three, epsilon),
        ]
    ).mean()

    percentage_error = 100 * (np.exp(log_accuracy_ratio_mean) - 1)

    return percentage_error.round(
        decimals=2
    )

'''
def main():
    answers_filename = sys.argv[1]
    responses_filename = sys.argv[2]

    answers = load_answers(answers_filename)
    responses = load_answers(responses_filename)

    print(get_smoothed_mean_log_accuracy_ratio(answers, responses))


if __name__ == '__main__':
    main()
'''

"\ndef main():\n    answers_filename = sys.argv[1]\n    responses_filename = sys.argv[2]\n\n    answers = load_answers(answers_filename)\n    responses = load_answers(responses_filename)\n\n    print(get_smoothed_mean_log_accuracy_ratio(answers, responses))\n\n\nif __name__ == '__main__':\n    main()\n"

In [0]:
x = pd.DataFrame({'hour_end': validate['hour_end'],
                  'hour_start': validate['hour_start'],
                  'time_delta': validate['hour_end'] - validate['hour_start'],
                  'count_publishers': validate['publishers'].apply(lambda x: len(x.split(','))),
                  'cpm': validate['cpm'],
                  'audience_size': validate['audience_size'],
                  'count_of_users_on_publ': count_of_users_on_publ,
                  'p': validate['audience_size'] / count_of_users_on_publ,
                  'mean_cmp': mean_cmp,
                  'diff': mean_cmp - validate['cpm'],
                  'sub': mean_cmp/validate['cpm'],
                  })
y = validate_answers

In [0]:
x.head()

,hour_end,hour_start,time_delta,count_publishers,cpm,audience_size,count_of_users_on_publ,p,mean_cmp,diff,sub
0,1153,1058,95,2,220.0,1906,4598,0.414528,253.183282,33.183282,1.150833
1,1301,1295,6,2,312.0,1380,10967,0.125832,173.083709,-138.916291,0.554755
2,1249,1229,20,6,70.0,888,34691,0.025597,197.274887,127.274887,2.818213
3,1377,1295,82,2,240.0,440,10146,0.043367,195.917368,-44.082632,0.816322
4,990,752,238,4,262.0,1476,42564,0.034677,234.729084,-27.270916,0.895913


In [0]:
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler

scores = []


model_1 = CatBoostRegressor(n_estimators=5000, 
                        depth=5,
                        l2_leaf_reg=0.1,
                        learning_rate=0.01,
                        )
model_2 = CatBoostRegressor(n_estimators=5000, 
                        depth=5,
                        l2_leaf_reg=0.1,
                        learning_rate=0.01,
                        )
model_3 = CatBoostRegressor(n_estimators=5000, 
                        depth=5,
                        l2_leaf_reg=0.1,
                        learning_rate=0.01,
                        )


iters1 = []
iters2 = []
iters3 = []

for (train_index, valid_index) in KFold(n_splits=5).split(x, y):
  scaler = MinMaxScaler()
  X_train, X_valid = x.iloc[train_index], x.iloc[valid_index]
  y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]

  model_1.fit(X_train, y_train['at_least_one'], eval_set=[(X_valid, y_valid['at_least_one'])], verbose=0, early_stopping_rounds=200)
  model_2.fit(X_train, y_train['at_least_two'], eval_set=[(X_valid, y_valid['at_least_two'])], verbose=0, early_stopping_rounds=200)
  model_3.fit(X_train, y_train['at_least_three'], eval_set=[(X_valid, y_valid['at_least_three'])], verbose=0, early_stopping_rounds=200)

  a = model_1.predict(X_valid)
  b = model_2.predict(X_valid)
  c = model_3.predict(X_valid)

  a[a<0]=0
  b[b<0]=0
  c[c<0]=0
  b[a<b] = a[a<b]/2
  c[b<c] = b[b<c]/2
  c[a<c] = a[a<c]/4

  iters1.append(model_1.best_iteration_)
  iters2.append(model_2.best_iteration_)
  iters3.append(model_3.best_iteration_)
  print(model_1.best_iteration_, model_2.best_iteration_, model_3.best_iteration_)
  res  = pd.DataFrame({'at_least_one':a, 'at_least_two':b, 'at_least_three':c})
  scores.append(get_smoothed_mean_log_accuracy_ratio(res, y_valid.reset_index()))
  print(get_smoothed_mean_log_accuracy_ratio(res, y_valid.reset_index()))
print(np.mean(scores))

2064 3992 1574
20.39
2794 2125 3258
18.4
487 2349 1153
26.98
3304 4015 2683
23.69
1392 4126 4992
24.19
22.729999999999997


In [0]:
print(np.mean(iters1))
print(np.mean(iters2))
print(np.mean(iters3))

2008.2
3321.4
2732.0


In [0]:
i1 = int(np.mean(iters1))-100
i2 = int(np.mean(iters2))-100
i3 = int(np.mean(iters3))-100
model_1 = CatBoostRegressor(n_estimators=i1, 
                        max_depth=5,
                        l2_leaf_reg=0.1,
                        learning_rate=0.01,
                        verbose=0)
model_2 = CatBoostRegressor(n_estimators=i2, 
                        max_depth=5,
                        l2_leaf_reg=0.1,
                        learning_rate=0.01,
                        verbose=0)
model_3 = CatBoostRegressor(n_estimators=i3, 
                        max_depth=5,
                        l2_leaf_reg=0.1,
                        learning_rate=0.01,
                        verbose=0)


In [0]:
model_1.fit(x, y['at_least_one'])
model_2.fit(x, y['at_least_two'])
model_3.fit(x, y['at_least_three'])

In [0]:
x.columns[np.argsort(model_3.feature_importances_)[::-1]]

Index(['time_delta', 'p', 'diff', 'sub', 'count_of_users_on_publ', 'cpm',
       'mean_cmp', 'audience_size', 'hour_end', 'hour_start',
       'count_publishers'],
      dtype='object')

In [0]:
%%time
a = model_1.predict(x)
b = model_2.predict(x)
c = model_3.predict(x)

a[a<0]=0
b[b<0]=0
c[c<0]=0
b[a<b] = a[a<b]/2
c[b<c] = b[b<c]/2
c[a<c] = a[a<c]/4
res  = pd.DataFrame({'at_least_one':a, 'at_least_two':b, 'at_least_three':c})
print(get_smoothed_mean_log_accuracy_ratio(res, y))

10.61
CPU times: user 19 ms, sys: 925 µs, total: 19.9 ms
Wall time: 14.4 ms


In [0]:
import pickle

pickle.dump(model_1, open("/vkcup/My Drive/Colab Notebooks/vkcup2/data/model1.pkl", "wb"))
pickle.dump(model_2, open("/vkcup/My Drive/Colab Notebooks/vkcup2/data/model2.pkl", "wb"))
pickle.dump(model_3, open("/vkcup/My Drive/Colab Notebooks/vkcup2/data/model3.pkl", "wb"))